In [ ]:
'''
精准率和召回率这两个指标有时精准率低一些有时召回率低一些，有时可能都低。那么实际中用哪个指标比较好呢？这一般和应用场景有关，对于有些场景，我们更注重精准率，比如股票预测，假设预测的是一个二分类问题：股票会升还是降，显然为了利润我们关注的是升（即上升为类1），为什么这种情况下精准率指标更好呢？因为精准率是所有分类为1的预测中有多少是正确的，对本例也就是预测未来股票上升有多少是对的，这更复合我们的利润最大决策。而召回率是实际上升的股票中我们预测对了多少，基于风险投资理念，有很多股票会上升的时刻，我们就算落掉一些也是没有关系的，没有投资进去也就没有损失，更重要的是我们的决策中有多少能赚钱，所以在这种场景下，精准率更好。
而如果在医疗领域，则是召回率更加重要，也就是要能在实际得病的人中尽量预测的更加准确，我们不想漏掉任何真正患病的人，这样才更有可能挽回一些人的生命，而精准率低些（没病的被预测为有病）并不会导致特别严重的后果，只是进行了一些过度医疗。
'''

In [ ]:
from sklearn import metrics
import numpy as np
import pandas as pd


In [ ]:
# 符合wikipedia的计算
# https://en.wikipedia.org/wiki/Discounted_cumulative_gain

true_relevance = np.asarray([[3, 2, 3, 0, 1, 2, 3, 2]])
scores = np.asarray([[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]])
metrics.ndcg_score(true_relevance, scores, k=6)

In [ ]:
# 以jd_id聚类
# 对于每一个jd_id，找到对应的cv_id列表，进而找到cv_id列表对应的real_label和pred_score
# 将real_label和pred_score送入ndcg_score()

In [ ]:
# 读取对测试集的预测。测试集的规模14627。
model_label_pred_label = pd.read_pickle('../data_20220831/model_label_pred_label.pkl')

model_label_pred_label

In [ ]:
# 对列进行重命名
model_label_pred_label = model_label_pred_label.rename(columns={'model_label': 'real_label', 0: 'prob_0', 1: 'prob_1'})
model_label_pred_label

In [ ]:
# 对全集计算ndcg，过于优秀
true_relevance = np.asarray([model_label_pred_label['real_label']])
scores = np.asarray([model_label_pred_label['prob_1']])
metrics.ndcg_score(true_relevance, scores, k=2000)

In [ ]:
model_label_pred_label.columns

In [ ]:
model_label_pred_label.groupby('jd_id').count()


In [ ]:
model_label_pred_label.groupby('jd_id').groups


In [ ]:
model_label_pred_label[model_label_pred_label['jd_id']=='2c9207157b9b9199017ba610cfe10a9d']

In [ ]:
model_label_pred_label.iloc[[7894, 12887, 13086]]

In [ ]:
model_label_pred_label.iloc[[7894]]['prob_1'].__class__

In [ ]:
# jd_id聚类的ndcg
for k in range(1, 6):

    ndcg_score_list = []
    for jd_id, row_list in model_label_pred_label.groupby('jd_id').groups.items():
        jd_cv_list_df = model_label_pred_label.iloc[row_list]

        true_relevance = list(jd_cv_list_df['real_label'])
        scores = list(jd_cv_list_df['prob_1'])
        
        # 注意这里的true_relevance和scores的长度若为1，会报错。故都补0，不影响计算。
        true_relevance.append(0)
        scores.append(0)

        # list转array
        true_relevance = np.asarray([true_relevance])
        scores = np.asarray([scores])
        
        # 计算ndcg@k
        ndcg_score = metrics.ndcg_score(true_relevance, scores, k=k)
        ndcg_score_list.append(ndcg_score)

    ndcg_score_list = np.array(ndcg_score_list)
    print('ndcg@{}: {}'.format(k, ndcg_score_list.mean()))

In [ ]:
a = np.asarray([1,2,3])
a = np.append(a, 4)
a

In [ ]:
# cv_id聚类的ndcg
for k in range(1, 6):

    ndcg_score_list = []
    for jd_id, row_list in model_label_pred_label.groupby('cv_id').groups.items():
        jd_cv_list_df = model_label_pred_label.iloc[row_list]

        true_relevance = list(jd_cv_list_df['real_label'])
        scores = list(jd_cv_list_df['prob_1'])
        
        # 注意这里的true_relevance和scores的长度若为1，会报错。故都补0，不影响计算。
        true_relevance.append(0)
        scores.append(0)

        # list转array
        true_relevance = np.asarray([true_relevance])
        scores = np.asarray([scores])
        
        # 计算ndcg@k
        ndcg_score = metrics.ndcg_score(true_relevance, scores, k=k)
        ndcg_score_list.append(ndcg_score)

    ndcg_score_list = np.array(ndcg_score_list)
    print('ndcg@{}: {}'.format(k, ndcg_score_list.mean()))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

sns.set()
f,ax = plt.subplots()
y_true = [0,0,1,2,1,2,0,2,2,0,1,1]
y_pred = [1,0,1,2,1,0,0,2,2,0,1,1]
C2 = confusion_matrix(y_true,y_pred,labels=[0,1,2])
#打印 C2
print(C2)
sns.heatmap(C2,annot=True,ax=ax) #画热力图

#ax.set_titile('confusion matrix') #标题
ax.set_xlabel('predict') #x 轴
ax.set_ylabel('true') #y 轴

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

sns.set()
f,ax = plt.subplots()

y_true = np.asarray(model_label_pred_label['real_label'])
y_pred = np.asarray(model_label_pred_label['pred_label'])

C2 = confusion_matrix(y_true,y_pred)
print(C2)

# 画热力图
sns.heatmap(C2,annot=True,ax=ax) 
ax.set_title('confusion matrix') 
ax.set_xlabel('predict') 
ax.set_ylabel('true') 

In [ ]:
6874/8347

In [ ]:
from sklearn.metrics import classification_report
 
print(classification_report(y_true, y_pred))

In [ ]:
plt.matshow(C2, cmap=plt.cm.gray)